In [1]:
import os
import yfinance as yf
from langchain.tools import BaseTool
from typing import Any, Type
from pydantic import BaseModel, Field
#from langchain.tools.render import format_tool_to_openai_function
from langchain.agents import initialize_agent
from typing import Optional
from langchain.chat_models import ChatOpenAI
from langchain.agents import AgentType


In [44]:
def get_stock_price(stock_ticker):   
    ticker = yf.Ticker(ticker=stock_ticker)
    current_price = ticker.history(period="1d")
    return round(current_price['Close'].iloc[-1], 2)

# Test the function
get_stock_price('AAPL')

193.58

In [3]:
def get_recommendations(recommend_stock_ticker):
     company = yf.Ticker(ticker=recommend_stock_ticker)
     return company.info['recommendationKey']


#### Test recommendations 

In [ ]:
# Test on 12/2023
print(get_recommendations('AAPL'))
print(get_recommendations('SHOP'))

print(get_recommendations('MO'))
print(get_recommendations('BYN')) #Banyan Acquisition Corp
#underperform, buy, hold

In [25]:
class StockPriceCheckInput(BaseModel):
    """Input to check Stock price."""

    stock_ticker: str = Field(description="Ticker symbol for stock")

In [27]:
class StockPriceTool(BaseTool):
    name = "get_stock_price"
    description = """
    Useful to find the stock price of a company and also suggest to buy number of stocks if asked. You should input the stock ticker.
    """
    
    def _run(self, stock_ticker: str):
        stock_price = get_stock_price(stock_ticker)
        return stock_price

    def _arun(self, stock_ticker: str):
        raise NotImplementedError()  
    
    args_schema: Type[BaseModel] = StockPriceCheckInput   

In [29]:
class RecommendationStockInput(BaseModel):
    """Input to the stock recommendations"""

    recommend_stock_ticker: str = Field(description="Ticker symbol for stock")

In [30]:
class RecommendationStockTool(BaseTool):
    name = "get_recommendations"
    description = "Useful to find recommendations to buy, hold if you have stocks or not to buy, You should input the stock ticker which is used by yahoo finance API"
    
    def _run(self, recommend_stock_ticker: str):
        stock_recommended_to_buy = get_recommendations(recommend_stock_ticker)
        return stock_recommended_to_buy

    def _arun(self, recommend_stock_ticker: str):
        raise NotImplementedError()  
    
    args_schema: Type[BaseModel] = RecommendationStockInput   

### Tools as openAI functions

In [31]:
stock_tools = [StockPriceTool(), RecommendationStockTool()]

### Langchain Agent

In [45]:
from langchain.schema import SystemMessage
llm = ChatOpenAI( api_key=os.environ['API_Key'], 
                   temperature=0 )

agent_kwargs = {
    "system_message": SystemMessage(content="You'er stock broker analyze the stock using tools and recommend to buy. You can access the tools.")
}

# Add prompt
open_ai_agent = initialize_agent(
                    tools = stock_tools, 
                    model = 'gpt-3.5-turbo',
                    llm=llm, 
                    agent = AgentType.OPENAI_MULTI_FUNCTIONS, 
                    agent_kwargs = agent_kwargs,
                    max_iterations = 2,
                    verbose=True)

### Get stock price.
LLM's are smart to convert company name to stock ticker

In [33]:
agent_results = open_ai_agent.run("what is the latest price of Amazon stock")
print(agent_results)



> Entering new AgentExecutor chain...

Invoking: `get_stock_price` with `{'stock_ticker': 'AMZN'}`


153.38The latest price of Amazon stock is $153.38.

> Finished chain.
The latest price of Amazon stock is $153.38.


### Get recommendation from LLM to buy a stock.
LLM's will be using yahoo finance api's recommendation

In [49]:
agent_results = open_ai_agent.run("what is the latest stock price of Beyond Meat, Inc, is it recommended to buy ?")
agent_results



> Entering new AgentExecutor chain...

Invoking: `get_stock_price` with `{'stock_ticker': 'BYND'}`


8.72
Invoking: `get_recommendations` with `{'recommend_stock_ticker': 'BYND'}`


underperformThe latest stock price of Beyond Meat, Inc (BYND) is $8.72. However, based on the recommendations, it is suggested to "underperform" this stock. Therefore, it is not recommended to buy Beyond Meat, Inc at the moment.

> Finished chain.


'The latest stock price of Beyond Meat, Inc (BYND) is $8.72. However, based on the recommendations, it is suggested to "underperform" this stock. Therefore, it is not recommended to buy Beyond Meat, Inc at the moment.'

### Number of stocks you can buy
Check to see if LLM can suggest the number of stocks to buy

In [50]:
agent_results = open_ai_agent.run("I am planning to invest $5000 in apple stock, how may stocks can I buy and is it recommended to invest in apple stocks")
agent_results



> Entering new AgentExecutor chain...

Invoking: `get_stock_price` with `{'stock_ticker': 'AAPL'}`


193.58
Invoking: `get_recommendations` with `{'recommend_stock_ticker': 'AAPL'}`


buyThe current price of Apple stock (AAPL) is $193.58. With an investment of $5000, you can buy approximately 25 shares of Apple stock.

As for the recommendation, the tool suggests buying Apple stock.

> Finished chain.


'The current price of Apple stock (AAPL) is $193.58. With an investment of $5000, you can buy approximately 25 shares of Apple stock.\n\nAs for the recommendation, the tool suggests buying Apple stock.'

Without prompt, 

{input} : I am planning to invest $5000 in apple stock, how may stocks can I buy and is it recommended to invest in apple stocks

'The current price of Apple stock (AAPL) is $193.6. With an investment of $5000, you can buy approximately 25.84 shares of Apple stock.\n\nAs for the recommendation, the tool suggests buying Apple stock.'

With prompting

{input}: I am planning to invest $5000 in apple stock, how may stocks can I buy and is it recommended to invest in apple stocks

'The current price of Apple stock (AAPL) is $193.58. With an investment of $5000, you can buy approximately 25 shares of Apple stock.\n\nAs for the recommendation, the tool suggests buying Apple stock.'

### Invest money and check how may stock to buy

In [51]:
agent_results = open_ai_agent.run("Is it recommended to buy apple stock ? if so how much do I need to invest for 25 stocks")



> Entering new AgentExecutor chain...



Invoking: `get_recommendations` with `{'recommend_stock_ticker': 'AAPL'}`


buy
Invoking: `get_stock_price` with `{'stock_ticker': 'AAPL'}`


193.58Based on the recommendation, it is recommended to buy Apple (AAPL) stock. The current price of Apple stock is $193.58.

To calculate how much you need to invest for 25 stocks, you can multiply the stock price by the number of stocks:

Investment amount = Stock price * Number of stocks
Investment amount = $193.58 * 25
Investment amount = $4,839.50

Therefore, you would need to invest approximately $4,839.50 for 25 stocks of Apple.

> Finished chain.


In [52]:
print(agent_results)

Based on the recommendation, it is recommended to buy Apple (AAPL) stock. The current price of Apple stock is $193.58.

To calculate how much you need to invest for 25 stocks, you can multiply the stock price by the number of stocks:

Investment amount = Stock price * Number of stocks
Investment amount = $193.58 * 25
Investment amount = $4,839.50

Therefore, you would need to invest approximately $4,839.50 for 25 stocks of Apple.


### Try to buy underperformed stock

In [53]:
agent_results = open_ai_agent.run("I am planning to invest $1000 in 'Beyond Meat, Inc' stock, is it recommended to invest in these stock if so how many can I buy")
agent_results



> Entering new AgentExecutor chain...

Invoking: `get_recommendations` with `{'recommend_stock_ticker': 'BYND'}`


underperform
Invoking: `get_stock_price` with `{'stock_ticker': 'BYND'}`


8.72Based on the recommendation, the stock for 'Beyond Meat, Inc' (BYND) is currently rated as "underperform". 

The current stock price for BYND is $8.72.

Since the stock is rated as underperform, it may not be recommended to invest in this stock at the moment. However, the final decision is up to you. If you still want to invest $1000 in BYND, you can calculate the number of shares you can buy by dividing the investment amount by the stock price:

Number of shares = $1000 / $8.72 = 114.68

Since you cannot buy fractional shares, you can buy a maximum of 114 shares of BYND with your $1000 investment.

> Finished chain.


'Based on the recommendation, the stock for \'Beyond Meat, Inc\' (BYND) is currently rated as "underperform". \n\nThe current stock price for BYND is $8.72.\n\nSince the stock is rated as underperform, it may not be recommended to invest in this stock at the moment. However, the final decision is up to you. If you still want to invest $1000 in BYND, you can calculate the number of shares you can buy by dividing the investment amount by the stock price:\n\nNumber of shares = $1000 / $8.72 = 114.68\n\nSince you cannot buy fractional shares, you can buy a maximum of 114 shares of BYND with your $1000 investment.'

In [54]:
print(agent_results)

Based on the recommendation, the stock for 'Beyond Meat, Inc' (BYND) is currently rated as "underperform". 

The current stock price for BYND is $8.72.

Since the stock is rated as underperform, it may not be recommended to invest in this stock at the moment. However, the final decision is up to you. If you still want to invest $1000 in BYND, you can calculate the number of shares you can buy by dividing the investment amount by the stock price:

Number of shares = $1000 / $8.72 = 114.68

Since you cannot buy fractional shares, you can buy a maximum of 114 shares of BYND with your $1000 investment.
